In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.neighbors import KNeighborsRegressor
from sklearn import model_selection

In [3]:
donnees = pd.read_csv("df_vst_group8.csv",decimal='.')

# Conversion de la colonne 'Heures' en type heure
donnees['date'] = pd.to_datetime(donnees['date'])

# Extraction de l'heure, des jours, des jours de semaines, des mois et des années dans des colonnes séparées
donnees['Heure'] = donnees['date'].dt.time.apply(lambda x: x.hour)
donnees['jour'] = donnees['date'].dt.date
donnees['weekday'] = donnees['date'].dt.weekday
donnees['month']= donnees['date'].dt.month
donnees['year'] = donnees['date'].dt.year


# Calcul de la température moyenne par jour
temperature_moyenne = donnees.groupby('jour')['temp'].mean().reset_index()
temperature_moyenne = temperature_moyenne.rename(columns={'temp': 'temp_moyenne'})

# Fusion des températures moyennes avec le dataframe d'origine
donnees_merged = pd.merge(donnees, temperature_moyenne, on='jour')

# Remplacement des valeurs de température par la température moyenne
donnees_merged['temp'] = donnees_merged['temp_moyenne']

# Suppression des colonnes intermédiaires
donnees_merged = donnees_merged.drop([ 'temp_moyenne'], axis=1)

donnees = donnees_merged

# Extraction des jours à prédire 
X_pred = donnees.query('index > 25489')

# Supression des lignes contenant NaN 

donnees.dropna(inplace=True)

print(donnees)

                     date      mean       temp  Heure        jour  weekday  \
0     2009-07-15 00:00:00  0.359623  14.955208      0  2009-07-15        2   
1     2009-07-15 00:30:00  0.300786  14.955208      0  2009-07-15        2   
2     2009-07-15 01:00:00  0.246548  14.955208      1  2009-07-15        2   
3     2009-07-15 01:30:00  0.211129  14.955208      1  2009-07-15        2   
4     2009-07-15 02:00:00  0.194699  14.955208      2  2009-07-15        2   
...                   ...       ...        ...    ...         ...      ...   
24859 2010-12-14 21:30:00  0.901088   3.307292     21  2010-12-14        1   
24860 2010-12-14 22:00:00  0.837083   3.307292     22  2010-12-14        1   
24861 2010-12-14 22:30:00  0.774550   3.307292     22  2010-12-14        1   
24862 2010-12-14 23:00:00  0.699087   3.307292     23  2010-12-14        1   
24863 2010-12-14 23:30:00  0.575146   3.307292     23  2010-12-14        1   

       month  year  
0          7  2009  
1          7  2009  


In [4]:


# Création de la matrice des données servant à la prévision 
X = donnees[['Heure','temp','weekday']]

y = donnees['mean']

max_temp = donnees['temp'].max()
donnees['temp'] = donnees['temp']/max_temp
donnees['Heure'] = donnees['Heure']*8/24 # 8 est un coefficient de poids sur l'heure, et le 24 un coefficient de normalisation 
donnees['weekday'] = donnees['weekday']*2/6
# Séparation des données en test et train 

X_train = donnees.query('(month > 7 and year == 2009) or (month < 5) or (month > 7 and month < 11 and year == 2010) ')[['Heure','temp','weekday']]
X_test =  donnees.query('month > 10 and year == 2010')[['Heure','temp','weekday']]
Y_train = donnees.query('(month > 7 and year == 2009) or (month < 5) or (month > 7 and month < 11 and year == 2010) ')[['mean']]
Y_test =  donnees.query('month > 10 and year == 2010')[['mean']]

In [7]:

from sklearn import model_selection


# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[19,20,22,23,25,26,27,29,30,31,32,33,34,35,36,37,38,39,40,45,48,50,52,54,56,60,62,64,66,67,68]}


# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(
    KNeighborsRegressor(), # un classifieur kNN
    param_grid,     # hyperparamètres à tester
    cv=5,           # nombre de folds de validation croisée
)

# Optimiser ce classifieur sur le jeu d'entraînement

clf.fit(X_train, Y_train)




GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [19, 20, 22, 23, 25, 26, 27, 29, 30, 31,
                                         32, 33, 34, 35, 36, 37, 38, 39, 40, 45,
                                         48, 50, 52, 54, 56, 60, 62, 64, 66, 67, ...]})

In [8]:
nb_neighbors = clf.best_params_['n_neighbors']
print(nb_neighbors)

60


In [38]:
#Entrainement du modèle avec kNN


model = KNeighborsRegressor(nb_neighbors, weights='distance')

model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

print(model.score(X_test, Y_test))

0.8941197900125557


In [39]:
#Mean Absolute Percentage Error

mape = np.mean(np.abs((Y_test - Y_pred)/Y_test))*100
print(mape)

mean    11.701333
dtype: float64


c:\Users\laure\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
